In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Dense, Dropout, Bidirectional, GRU, Concatenate
from keras.models import Model
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from dateutil import parser
import re
import math
from keras import backend
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import os, datetime
from tensorflow.keras.callbacks import Callback
import tensorflow as tf
from keras.models import load_model
import json

In [2]:
from flask import Flask, jsonify, request
import joblib
from datetime import datetime

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading the models

In [5]:
final_model = load_model("/content/drive/MyDrive/CaseStudy2/final_model.h5", compile=False)

In [6]:
test_model = load_model("/content/drive/MyDrive/CaseStudy2/test_model.h5", compile=False)

In [12]:
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 21)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 21, 10)       100000      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 21, 384), (N 235008      embedding[0][0]                  
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 384)          0           bidirectional[0][1]              
                                                                 bidirectional[0][2]          

In [13]:
test_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 10)     100000      input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
gru_1 (GRU)                     [(None, None, 384),  456192      embedding_1[0][0]                
                                                                 input_3[0][0]              

In [15]:
max_len_size_vocab = 10000

with open('/content/drive/MyDrive/CaseStudy2/word_dict-final.json') as f:
    dict_words = json.load(f)
    toknizer = Tokenizer(filters='', num_words=max_len_size_vocab)
    toknizer.word_index = dict_words

In [16]:
max_len_in = 21
max_len_out = 20
def test_tokenize_data(data):
    data = data.lower()
    data = '<start> ' + data + ' <end>'
    data_tf = toknizer.texts_to_sequences([data])
    data_tf = pad_sequences(data_tf, maxlen=max_len_in, padding="post")
    return data_tf

In [17]:
integer2words = map(reversed, toknizer.word_index.items())
integer2words = dict(integer2words)
def get_testoutput(query_point):
    st = final_model.predict(query_point)
    sequence_output = np.zeros((1, 1))
    sequence_output[0, 0] = toknizer.word_index['<start>']
    present = "<start>"
    sen_final = ''
    k = 0
    while present != "<end>" and k < (max_len_out - 1):
        tkns_final, st_h = test_model.predict([sequence_output, st])

        tkn_present = np.argmax(tkns_final[0, 0])

        if (tkn_present == 0):
          break;

        present = integer2words[tkn_present]

        sen_final += ' ' + present
        sequence_output[0, 0] = tkn_present
        st = st_h
        k += 1

    return sen_final

In [18]:
def getWords(w):
    res=''
    if w!=0:
      res = integer2words[w]
    else:
      res = ''
    return res

def tkns2seq(inputs):
  
  sentnces = list(map(getWords, inputs))
  result = ' '.join(sentnces)
  return result

In [29]:
query_points = ['shall we connect in', 
                'have a', 
                'let me know',
                'please find my',
                'please find my resume',
                'I am',
                'can we',
                'As discussed can we connect this',
                'I will finalize and submit the project in']

In [30]:
for i in query_points:
    result_qp = get_testoutput(test_tokenize_data(i))
    print(f"QUERY_POINT: {i}  NEXT SEQUENCE: {result_qp}")

QUERY_POINT: shall we connect in  NEXT SEQUENCE:  the office on friday. thanks <end>
QUERY_POINT: have a  NEXT SEQUENCE:  good <end>
QUERY_POINT: let me know  NEXT SEQUENCE:  if you have any questions. <end>
QUERY_POINT: please find my  NEXT SEQUENCE:  attached links report. jerry graves <end>
QUERY_POINT: please find my resume  NEXT SEQUENCE:  for the attached ngi chicago employees. <end>
QUERY_POINT: I am  NEXT SEQUENCE:  going to be out of the office until <end>
QUERY_POINT: can we  NEXT SEQUENCE:  send me the latest gtc's for the game. thanks russell <end>
QUERY_POINT: As discussed can we connect this  NEXT SEQUENCE:  week. <end>
QUERY_POINT: I will finalize and submit the project in  NEXT SEQUENCE:  the interstate pipelines? <end>
